In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics

In [2]:
# load the data
stv = pd.read_hdf('sales_train_validation.h5')
cal = pd.read_hdf('calendar.h5')
sp = pd.read_hdf('sell_prices.h5')
# create a list for date column names
d_cols = ['d_' + str(i) for i in range(1, 1914)]

In [3]:
# store data in dtypes using less memory
cal['wm_yr_wk'] = cal['wm_yr_wk'].astype('int16')
cal[['wday', 'month']] = cal[['wday', 'month']].astype('int8')

In [4]:
# create dataframe for sales record
stv.index = stv['id'].apply(lambda x: x[:-11])
sales_by_depts = stv[d_cols+['dept_id', 'id']].groupby('dept_id')
temp = sales_by_depts.get_group('HOBBIES_1')
temp = temp.drop(['dept_id', 'id'], axis=1).T
temp['date'] = pd.to_datetime(cal['date'][:1913].values, format='%Y-%m-%d')
# add Walmart year and week infomation
temp['wm_yr_wk'] = cal['wm_yr_wk'][:1913].values
# add holiday information
temp['event'] = cal['event_name_1'][:1913].values
temp['event'] = temp['event'].replace('nan', 0)
indices = temp['event'][temp['event']!=0].index
temp['event'].loc[indices] = list(range(1, 155))

d:\python\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
# melt down the dataframe while keeping the date, wm_yr_wk
# and event label for each item
df_sales = pd.melt(temp, id_vars=['date', 'wm_yr_wk', 'event'])
df_sales.rename(columns={'value': 'sales'}, inplace=True)

In [6]:
# create a dataframe for price record
sp['dept_id'] = sp['item_id'].apply(lambda x: x[:-4])
sp['id'] = sp['item_id'] + '_' + sp['store_id']
sp = sp.drop(['store_id', 'item_id'], axis=1)
sp['wm_yr_wk'] = sp['wm_yr_wk'].astype('int16')
prices_by_depts = sp.groupby('dept_id')
df_prices = prices_by_depts.get_group('HOBBIES_1')

In [7]:
# delete some data to save memory
del temp; del stv; del sp; del cal

In [8]:
# concatenate sales and price dataframes
# make the index unique
df_sales.index = df_sales['wm_yr_wk'].astype('str') + '_'  + df_sales['id']
df_prices.index = df_prices['wm_yr_wk'].astype('str') + '_'  + df_prices['id']
df_sales['sell_price'] = df_prices['sell_price']
# fill the null with 0
df_sales['sell_price'].fillna(value=0, inplace=True)
# reset index
df_sales.reset_index(drop=True, inplace=True)

In [9]:
# add day of the month information
df_sales['day_of_month'] = df_sales['date'].dt.day
# add day of the week information
df_sales['day_of_week'] = df_sales['date'].dt.dayofweek

In [10]:
# delete useless data to save memory
del df_prices
df_sales.drop('wm_yr_wk', inplace=True, axis=1)
# reindex and drop useless columns
df_sales.index = df_sales['date']
df_sales.drop('date', inplace=True, axis=1)

In [12]:
# detect outliers
def find_outliers(series):
    outliers = (series - series.mean()) > 1.96 * series.std()
    return outliers
# replace the outliers with the maximum of normal values
def cap_values(series):
    outliers = find_outliers(series)
    maximum = series[~outliers].max()
    series[outliers] = maximum
    return series

# group the data by items' id and iterate over each item
items = df_sales.groupby('id')
df_sales['is_outlier'] = items.apply(lambda x: find_outliers(x['sales'])).values.ravel()
df_sales['sales_capped'] = items.apply(lambda x: cap_values(x['sales'])).values.ravel()
# return the natural logrithm of 1 plus the input array
df_sales['sales_log1p'] = np.log1p(df_sales['sales'])
df_sales['sales_capped_log1p'] = np.log1p(df_sales['sales_capped']) 

In [ ]:
# a function to show the process
# feel free to annotate it if it's unecessary for you
items_count = len(items)
def counter(func):
    temp = [int(items_count*i*0.1) for i in range(1, 11)]
    def wrapper(data, on):
        wrapper.count += 1
        if wrapper.count in temp:
            print(f'processing:{wrapper.count/items_count*100}%')
        return func(data, on)
    wrapper.count = 0
    return wrapper

In [13]:
# @counter
def ex_stats(data, on):
    # create a dict to collect all the stats
    stats = {'mean':[], 'std':[], 'median':[], 'max':[], 'min':[]}
    # iterate over groups and calculate precedent statistics
    alphas = [0.1, 0.25, 0.3, 0.5, 0.75, 1]
    stats.update({f'exp_{alpha}_mean':[] for alpha in alphas})
    # create groupby object
    shift = data[on].shift()
    roll = shift.expanding()
    stats['mean'].extend(roll.mean())
    stats['std'].extend(roll.std())
    stats['median'].extend(roll.median())
    stats['max'].extend(roll.max())
    stats['min'].extend(roll.min())
    # calulate ewm series with different alphas    
    for alpha in alphas:
        exp_roll = shift.ewm(alpha=alpha, adjust=False)
        stats[f'exp_{alpha}_mean'].extend(exp_roll.mean())
    for keys, values in stats.items():
        data[f'{on}_{keys}'] = values
    return data
# applying ex_stats to each item
df_sales = items.apply(lambda x: ex_stats(data=x, on='sales'))
items = df_sales.groupby('id')
df_sales = items.apply(lambda x: ex_stats(x, on='sales_capped'))
items = df_sales.groupby('id')
df_sales = items.apply(lambda x: ex_stats(x, on='sales_capped_log1p'))
# drop useless columns

In [14]:
df_sales.head()

,event,id,sales,sell_price,day_of_month,day_of_week,is_outlier,sales_capped,sales_log1p,sales_capped_log1p,...,sales_capped_log1p_std,sales_capped_log1p_median,sales_capped_log1p_max,sales_capped_log1p_min,sales_capped_log1p_exp_0.1_mean,sales_capped_log1p_exp_0.25_mean,sales_capped_log1p_exp_0.3_mean,sales_capped_log1p_exp_0.5_mean,sales_capped_log1p_exp_0.75_mean,sales_capped_log1p_exp_1_mean
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,0,HOBBIES_1_001_CA_1,0,0.0,29,5,False,0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-30,0,HOBBIES_1_001_CA_1,0,0.0,30,6,False,0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-31,0,HOBBIES_1_001_CA_1,0,0.0,31,0,False,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-01,0,HOBBIES_1_001_CA_1,0,0.0,1,1,False,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-02,0,HOBBIES_1_001_CA_1,0,0.0,2,2,False,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df_sales['items_id'] = df_sales['id'].str.slice(0, 13)
df_sales['store_id'] = df_sales['id'].str.slice(14)

In [19]:
to_drop = ['id', 'is_outlier', 'sales_capped', 'sales_capped_log1p']
df_sales = df_sales.drop(to_drop, axis=1)

In [16]:
# save the file
df_sales.to_hdf('HOBBIES_1.h5', key='df', mode='w', format='fixed')

d:\python\lib\site-packages\pandas\core\generic.py:2490: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_values] [items->Index(['event'], dtype='object')]

  pytables.to_hdf(


In [ ]:
# extract time series information
from scipy import optimize

# define a function to get the exponential weighted moving average series
def calc_shifted_ewm(series, alpha, adjust=True):
    # move the values to the next day
    # adjust == True means using weights (1-a)^2 
    return series.shift().ewm(alpha=alpha, adjust=adjust).mean()

## define an optimization function to select the best alpha
def find_best_signal(series, adjust=False, eps=10e-5):
    
    def find(alpha):
        # make sure that alpha is between 0 and 1
        shifted_ewm = calc_shifted_ewm(series=series, alpha=min(max(alpha, 0), 1), 
                                       adjust=adjust)
        # define a loss function
        loss = np.mean(np.power(series - shifted_ewm, 2))
        return loss
    
    # find the global minimum of the function
    # get the optimal alpha and pass it to ewm calculation function
    res = optimize.differential_evolution(func=find, bounds=[(0+eps, 1-eps)])
    best_alpha = res['x'][0]
    return calc_shifted_ewm(series=series, alpha=best_alpha, adjust=adjust)
2
# iterate over items and add ewmed sales series
groups = df_sales.groupby(['id', 'day_of_week'])
roll_sales_capped = groups.apply(lambda x: find_best_signal(x['sales_capped']))
df_sales['optimiazed_ewm_by_id_&_day_of_week'] = roll_sales_capped.values.ravel()
roll_sales_capped_log1p = groups.apply(lambda x: find_best_signal(x['sales_capped_log1p']))
df_sales['optimiazed_ewm_log1p_by_id_&_day_of_week'] = roll_sales_capped_log1p.values.ravel()